In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import r2_score
from data_utils import load_info, create_dataloaders, load_preprocessed_data

In [86]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim, dropout_prob=0.5):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.LeakyReLU(negative_slope=0.01))
            layers.append(nn.Dropout(dropout_prob))
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.network = nn.Sequential(*layers)
        self._initialize_weights()

    def forward(self, x):
        return self.network(x)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

In [87]:
def l1_regularization(model, l1_lambda):
    l1_norm = sum(p.abs().sum() for p in model.parameters())
    return l1_lambda * l1_norm

def train(model, train_loader, valid_loader, criterion, optimizer, scheduler, epochs, patience, l1_lambda):
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            inputs, targets = inputs.float(), targets.float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets) + l1_regularization(model, l1_lambda)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in train_loader:
                inputs, targets = inputs.float(), targets.float()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
        
        val_loss /= len(valid_loader.dataset)
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

        scheduler.step(val_loss)
        
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break

def test(model, test_loader):
    model.eval()
    targets_list = []
    outputs_list = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.float(), targets.float()
            outputs = model(inputs)
            
            targets_list.append(targets.numpy())
            outputs_list.append(outputs.numpy())

    outputs_list = np.concatenate(outputs_list)
    targets_list = np.concatenate(targets_list)

    r2 = r2_score(targets_list, outputs_list)
    print(f'R² score: {r2}\n\n')

In [88]:
input_data, target_data = load_preprocessed_data()
firm_info, _ = load_info()
train_loader, valid_loader, test_loader, _ = create_dataloaders(
    input_data, target_data, firm_info,
    train_date='2005-01-01', valid_date='2010-01-01', test_date='2015-11-01', batch_size=2000)

In [89]:
input_dim = input_data.shape[1] - 2
hidden_layers_list = [[128, 64, 32], [32, 16, 8], [128, 64], [64, 32], [32, 16], [16, 8]]
output_dim = 1
learning_rates = [0.05, 0.01]
epochs = 100
patience = 10
l1_lambdas = [1e-3]

In [90]:
for hidden_layers in hidden_layers_list:
    for learning_rate in learning_rates:
        for l1_lambda in l1_lambdas:
            print(f"Training with hidden_layers={hidden_layers}, learning_rate={learning_rate}, l1_lambda={l1_lambda}\n")
            model = NeuralNetwork(input_dim, hidden_layers, output_dim, dropout_prob=0.5)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

            train(model, train_loader, valid_loader, criterion, optimizer, scheduler, epochs, patience, l1_lambda)
            test(model, test_loader)

Training with hidden_layers=[128, 64, 32], learning_rate=0.05, l1_lambda=0.001

Epoch 1/100, Train Loss: 0.971981, Val Loss: 0.061118
Epoch 2/100, Train Loss: 0.258345, Val Loss: 0.061124
Epoch 3/100, Train Loss: 0.256588, Val Loss: 0.061207
Epoch 4/100, Train Loss: 0.259445, Val Loss: 0.061303
Epoch 5/100, Train Loss: 0.261571, Val Loss: 0.061347
Epoch 6/100, Train Loss: 0.263246, Val Loss: 0.061362
Epoch 7/100, Train Loss: 0.264492, Val Loss: 0.061346
Epoch 8/100, Train Loss: 0.071353, Val Loss: 0.061161
Epoch 9/100, Train Loss: 0.053893, Val Loss: 0.061157
Epoch 10/100, Train Loss: 0.053972, Val Loss: 0.061152
Epoch 11/100, Train Loss: 0.054020, Val Loss: 0.061145
Early stopping triggered
R² score: -0.01593107048059128


Training with hidden_layers=[128, 64, 32], learning_rate=0.01, l1_lambda=0.001

Epoch 1/100, Train Loss: 1.438741, Val Loss: 0.060968
Epoch 2/100, Train Loss: 0.226698, Val Loss: 0.061142
Epoch 3/100, Train Loss: 0.117184, Val Loss: 0.060841
Epoch 4/100, Train Loss:

KeyboardInterrupt: 